# Import and pre-definitions

In [1]:
import os
MAIN_PATH = r'/home/luis-felipe'
DATA_PATH = os.path.join(MAIN_PATH,'data')
PATH_MODELS = os.path.join(MAIN_PATH,'torch_models')
FIGS_PATH = os.path.join(MAIN_PATH,'results','figs')

In [2]:
import torch
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
# Define o computador utilizado como cuda (gpu) se existir ou cpu caso contrário
print(torch.cuda.is_available())
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
torch.set_default_dtype(torch.float64)
torch.manual_seed(42)
np.random.seed(42)

True


In [4]:
import sys
sys.path.insert(1, '..')
sys.path.insert(1, '../..')

import models
from utils import measures,metrics
from data_utils import upload_logits,split_data
import post_hoc

# Evaluate logits

In [5]:
DATASET = 'ImageNet'
VAL_SIZE = 0.1 #5000
METRIC = metrics.AURC

In [6]:
def normalized_entropy(logits:torch.tensor):
    return measures.entropy(logits).div(np.log(logits.size(-1)))
def sigma_sp(a:torch.tensor):
    return (a.exp()+1).log()
def T_hts(logits:torch.tensor,w,b):
    return sigma_sp(normalized_entropy(logits).log()*w+b)
def ats(logits:torch.tensor,T_fn = T_hts,**kwargs):
    return measures.MSP(logits.div(T_fn(logits,**kwargs).reshape(-1,1)))

In [7]:
def optimize_wb(logits:torch.tensor,risk,w_range,b_range, metric = METRIC):
    vals = []
    for w in w_range:
        vals_b = []
        for b in b_range:
            vals_b.append(metric(risk,ats(logits,T_hts,w=w,b=b)).item())
        vals.append(vals_b)
    w,b = np.unravel_index(np.argmin(vals),np.shape(vals))
    return w_range[w],b_range[b]

In [8]:
w_range = np.arange(-1,1,0.01)
b_range = np.arange(-2,1,0.01)

In [9]:
d_naurc = {}
d_baseline = {}
acc = {}

In [10]:
for model_arc in models.list_models():
    print(model_arc)
    #if model_arc in df['model'].values:
    #    print('ja foi')
    #    continue
    if 'quantized' in model_arc:
        print('quantized - skip')
        continue
    with torch.no_grad():
        try : logits_val,labels_val,logits_test,labels_test = split_data.split_logits(*upload_logits(model_arc,DATASET,PATH_MODELS, 
                            split = 'test', device = dev),VAL_SIZE)
        except: 
            print('cant find')
            continue
    logits_val = post_hoc.centralize(logits_val)
    logits_test = post_hoc.centralize(logits_test)
    risk_val = measures.wrong_class(logits_val,labels_val).float()
    risk_test = measures.wrong_class(logits_test,labels_test).float()
    acc[model_arc] = (1-risk_test.mean().item())

    w,b = optimize_wb(logits_val,risk_val,w_range,b_range)
    print(w,b)
    d_naurc[model_arc] = metrics.N_AURC(risk_test,ats(logits_test,T_hts,w=w,b=b))
    d_baseline[model_arc] =  metrics.N_AURC(risk_test,measures.MSP(logits_test))
models_list = list(acc.keys())

alexnet
-0.08999999999999919 0.2300000000000022
convnext_base
-0.27999999999999936 -0.5099999999999987
convnext_large
-0.23999999999999932 -0.6099999999999988
convnext_small
-0.24999999999999933 -0.849999999999999
convnext_tiny
-0.3099999999999994 -0.5099999999999987
deeplabv3_mobilenet_v3_large
cant find
deeplabv3_resnet101
cant find
deeplabv3_resnet50
cant find
densenet121
8.881784197001252e-16 0.3400000000000021
densenet161
0.07000000000000095 0.5500000000000025
densenet169
0.08000000000000096 0.7200000000000024
densenet201
0.08000000000000096 0.4100000000000019
efficientnet_b0
-0.1999999999999993 -0.10999999999999832
efficientnet_b1
-0.24999999999999933 -0.05999999999999828
efficientnet_b2
-0.2099999999999993 0.030000000000002025
efficientnet_b3
-0.14999999999999925 -0.2899999999999985
efficientnet_b4
-0.17999999999999927 -0.899999999999999
efficientnet_b5
-0.24999999999999933 -0.1999999999999984
efficientnet_b6
-0.15999999999999925 -0.16999999999999837
efficientnet_b7
-0.209999999

In [11]:
baseline = np.array([0.22344933, 0.29767382, 0.29555625, 0.29439124, 0.28604031,
       0.19153111, 0.19069721, 0.18937816, 0.18924556, 0.21095271,
       0.21944162, 0.23361707, 0.25291316, 0.30013695, 0.23502147,
       0.23039686, 0.25052851, 0.24555458, 0.28631002, 0.22839592,
       0.21536632, 0.22731528, 0.22279686, 0.22289863, 0.30586287,
       0.18318941, 0.32675117, 0.27949274, 0.21829896, 0.19292605,
       0.22706483, 0.32766483, 0.23726933, 0.26794277, 0.30517194,
       0.33082617, 0.23197287, 0.15433325, 0.27863751, 0.26171108,
       0.24323554, 0.23182883, 0.25692708, 0.24738116, 0.2291486 ,
       0.26318683, 0.25402228, 0.2000635 , 0.19133273, 0.32177478,
       0.42262309, 0.39333033, 0.26853085, 0.21921907, 0.19646585,
       0.28629977, 0.28216046, 0.23412546, 0.2231855 , 0.27783691,
       0.23290676, 0.21702466, 0.24943449, 0.23173071, 0.21784759,
       0.19313987, 0.18983575, 0.18984926, 0.1882965 , 0.18368938,
       0.1815334 , 0.18306352, 0.18351593, 0.2359024 , 0.22741867,
       0.16946717, 0.22462286, 0.24602972, 0.2764518 , 0.35852901,
       0.44010379, 0.14587205, 0.15570429, 0.16867164])

In [12]:
naurc = np.array(list(d_naurc.values()))

In [13]:
gain = baseline-naurc

In [14]:
gain

array([-3.23126468e-05,  1.26534402e-01,  1.28190476e-01,  1.28554785e-01,
        1.14452409e-01,  4.54463914e-05,  4.17907688e-04,  8.68755349e-05,
        4.55999641e-04,  3.08126179e-02,  4.27950657e-02,  5.39202126e-02,
        7.60635969e-02,  1.28134232e-01,  7.19161551e-02,  6.29376090e-02,
        9.15984284e-02,  8.04571305e-02,  1.16685071e-01,  6.75396945e-02,
        9.91595089e-03,  3.56252908e-02,  5.45042610e-02,  1.55295521e-02,
        1.02537986e-01, -3.12745423e-04,  1.35851360e-01,  8.19665074e-02,
        3.10904254e-02,  3.41798634e-04,  5.97887369e-02,  1.43406246e-01,
        7.06291801e-02,  8.96921072e-02,  1.15386885e-01,  1.41320543e-01,
        6.35808562e-02,  7.27595041e-03,  1.15702492e-01,  8.47110763e-02,
        8.06226531e-02,  5.66911212e-02,  6.65078999e-02,  6.74075357e-02,
        6.56283696e-02,  8.76383377e-02,  9.05321829e-02,  5.19374871e-04,
        3.03481914e-05,  1.47950227e-01,  2.49159034e-01,  2.23212529e-01,
        9.21719185e-02,  

In [15]:
np.mean(post_hoc.significant(gain, 0.01))

0.06281449017661803